# Week 2 - Overview of Lyapunov Stability Theory

**<ins>Motivation</ins>**

In this week, we introduce **Lyapunov’s Direct Method** as a powerful tool to analyze the stability of nonlinear systems.  
Unlike linearization methods (which only provide local insights), Lyapunov’s method allows us to reason about **global stability** properties without explicitly solving the system equations.

**<ins>Key Ideas</ins>**

- **Lyapunov’s Direct Method**:  
  Provides a systematic way to prove stability and convergence for nonlinear systems by constructing a scalar "energy-like" function.

- **Definiteness of Functions**:  
  The concept of *positive definite*, *negative definite*, and *semi-definite* functions is introduced.  
  This serves as the **building block** for Lyapunov’s direct method.

- **Candidate Lyapunov Functions**:  
  Convenient prototypes are presented for:
  - **Rate-error measures** (functions of angular velocity error).  
  - **State-error measures** (functions of attitude error).
 
**<ins>Key Ideas</ins>**

By the end of this week, you should be able to:
1. Apply Lyapunov’s direct method to argue **stability and convergence** on a range of dynamical systems.
2. Differentiate between a range of **nonlinear stability concepts** (e.g., stability, asymptotic stability, exponential stability).

In [1]:
import sympy as sp
sp.init_printing(use_latex='mathjax')

from IPython.display import display

import numpy as np

# 2.1 - Lyapunov's Direct Method

## 2.1.1 - Definiteness of Functions

**<ins>Positive/Negative Definite Function</ins>**

A scalar continuous function $V(x)$ is said to be **locally positive definite** about $x_r$ if:

- $x = x_r \;\;\Rightarrow\;\; V(x) = 0$
- There exists a $\delta > 0$ such that  
  $\forall x \in B_\delta(x_r) \;\;\Rightarrow\;\; V(x) > 0$  

Similarly, $V(x)$ is **locally negative definite** if:  
- $x = x_r \;\;\Rightarrow\;\; V(x) = 0$  
- $\forall x \in B_\delta(x_r) \;\;\Rightarrow\;\; V(x) < 0$  

**<ins>Positive/Negative Semi-Definite Function</ins>**

A scalar continuous function $V(x)$ is said to be **locally positive semi-definite** about $x_r$ if:

- $x = x_r \;\;\Rightarrow\;\; V(x) = 0$
- There exists a $\delta > 0$ such that  
  $\forall x \in B_\delta(x_r) \;\;\Rightarrow\;\; V(x) \geq 0$  

Similarly, $V(x)$ is **locally negative semi-definite** if:  
- $x = x_r \;\;\Rightarrow\;\; V(x) = 0$  
- $\forall x \in B_\delta(x_r) \;\;\Rightarrow\;\; V(x) \leq 0$  

A common prototype Lyapunov candidate:
$$
V(x, \dot{x}) = \tfrac{1}{2}x^2 + \tfrac{1}{2}\dot{x}^2
$$

This function is positive definite, since $V(x, \dot{x}) = 0$ only at $(x, \dot{x}) = (0,0)$, and $V(x, \dot{x}) > 0$ otherwise.

**<ins>Matrix Definiteness</ins>**

A matrix $[K]$ is said to be positive or negative (semi-)definite if for every state vector $x$:

$$
x^T[K]x
\begin{cases}
> 0 & \text{positive definite} \\
\geq 0 & \text{positive semi-definite} \\
< 0 & \text{negative definite} \\
\leq 0 & \text{negative semi-definite}
\end{cases}
$$

**<ins>The Intuition</ins>**

- Think of $V(x)$ like an **energy function**:  
  - Positive definite → strictly positive “energy” everywhere except at equilibrium.  
  - Semi-definite → “energy” is non-negative but may be flat in some directions.  

- For matrices $[K]$:  
  - Positive definite matrices act like **stiff springs**: they store positive energy no matter what direction you displace in.  
  - Semi-definite means there might be “flat” directions where no restoring energy appears.  

- These concepts matter because Lyapunov stability proofs rely on choosing $V(x)$ that behaves like an “energy bowl” centered at the equilibrium point.

In [20]:
def check_definiteness(obj, variables=None, equilibrium=None, sample_points=100):
    """
    Check definiteness of a matrix (NumPy) or a function (SymPy).
    
    Parameters
    ----------
    obj : numpy.ndarray or sympy.Expr
        The matrix or scalar function to check.
        
    variables : list of sympy.Symbol, optional
        Variables used in the function (only needed if obj is SymPy expression).
        
    equilibrium : list/tuple of floats, optional
        Equilibrium point around which to test (default = all zeros).
        
    sample_points : int
        Number of random samples near equilibrium for numeric testing (function case).
    
    Returns
    -------
    str : Definiteness classification
    """
    
    # Case 1: Matrix
    if isinstance(obj, np.ndarray):
        eigvals = np.linalg.eigvals(obj)
        print("Matrix:\n", obj)
        print(f"Eigenvalues: {eigvals}")
        if np.all(eigvals > 0):
            return "Positive definite"
        elif np.all(eigvals >= 0):
            return "Positive semi-definite"
        elif np.all(eigvals < 0):
            return "Negative definite"
        elif np.all(eigvals <= 0):
            return "Negative semi-definite"
        else:
            return "Indefinite"
    
    # Case 2: Symbolic function (expressed using SymPy)
    elif isinstance(obj, sp.Expr):
        display("Function V(x):", obj)
        if variables is None:
            raise ValueError("Must provide variables for SymPy function")
        if equilibrium is None:
            equilibrium = [0]*len(variables)
        
        # Check equilibrium point
        V_eq = obj.subs(dict(zip(variables, equilibrium)))
        if V_eq != 0:
            return f"Not a valid Lyapunov candidate (V(eq) = {V_eq})"
        
        # Sample near equilibrium
        vals = []
        for _ in range(sample_points):
            rand_point = np.random.uniform(-1, 1, len(variables)) * 0.5
            V_val = float(obj.subs(dict(zip(variables, rand_point))))
            vals.append(V_val)
        
        if all(v > 0 for v in vals):
            return "Positive definite (numerical check)"
        elif all(v >= 0 for v in vals):
            return "Positive semi-definite (numerical check)"
        elif all(v < 0 for v in vals):
            return "Negative definite (numerical check)"
        elif all(v <= 0 for v in vals):
            return "Negative semi-definite (numerical check)"
        else:
            return "Indefinite (numerical check)"
    
    else:
        raise TypeError("Input must be a NumPy matrix or SymPy expression")

# Example Usage:

# Case 1: Inputting a matrix
print("Case 1:")
K = np.array([[-1, 0], 
              [0, 0]])
print(check_definiteness(K))

print()

#Case 2: Inputting a function V(x)
print("Case 2:")
x, xdot = sp.symbols('x xdot')
V = 0.5*x**2 #+ 0.5*xdot**2
print(check_definiteness(V, variables=[x, xdot]))

Case 1:
Matrix:
 [[-1  0]
 [ 0  0]]
Eigenvalues: [-1.  0.]
Negative semi-definite

Case 2:


'Function V(x):'

     2
0.5⋅x 

Positive definite (numerical check)


In [4]:
# Concept Check 1 - Qn1
x1, x2 = sp.symbols('x1 x2')
V = 0.5*(x1**2 + x2**2)
print(check_definiteness(V, variables=[x1, x2]))

'Function V(x):'

      2         2
0.5⋅x₁  + 0.5⋅x₂ 

Positive definite (numerical check)


In [5]:
# Concept Check 1 - Qn2
x1, x2 = sp.symbols('x1 x2')
V = 0.5*(x1**2 - x2**2)
print(check_definiteness(V, variables=[x1, x2]))

'Function V(x):'

      2         2
0.5⋅x₁  - 0.5⋅x₂ 

Indefinite (numerical check)


In [6]:
# Concept Check 1 - Qn3
x1, x2 = sp.symbols('x1 x2')
V = sp.log(1 + x1**2 + x2**2)
print(check_definiteness(V, variables=[x1, x2]))

'Function V(x):'

   ⎛  2     2    ⎞
log⎝x₁  + x₂  + 1⎠

Positive definite (numerical check)


In [7]:
# Concept Check 1 - Qn4
x1, x2 = sp.symbols('x1 x2')
V = 0.5*(x1**2 + 4*x2**2)
print(check_definiteness(V, variables=[x1, x2]))

'Function V(x):'

      2         2
0.5⋅x₁  + 2.0⋅x₂ 

Positive definite (numerical check)


In [8]:
# Concept Check 1 - Qn5
x1, x2 = sp.symbols('x1 x2')
V = 0.5*(x1**2 + 4*x2**2) * sp.exp(-(x1**2 + 4*x2**2))
print(check_definiteness(V, variables=[x1, x2]))

'Function V(x):'

                         2       2
⎛      2         2⎞  - x₁  - 4⋅x₂ 
⎝0.5⋅x₁  + 2.0⋅x₂ ⎠⋅ℯ             

Positive definite (numerical check)


In [9]:
# Concept Check 1 - Qn6
K = np.array([[1.53947, -0.0422688, -0.190629], 
              [-0.0422688, 1.4759, 0.459006],
              [-0.190629, 0.459006, 1.48463]])
print(check_definiteness(K))

Matrix:
 [[ 1.53947   -0.0422688 -0.190629 ]
 [-0.0422688  1.4759     0.459006 ]
 [-0.190629   0.459006   1.48463  ]]
Eigenvalues: [1.99999822 1.50000363 0.99999815]
Positive definite


In [10]:
# Concept Check 1 - Qn7
K = np.array([[-0.984331, -1.10006, -0.478579], 
              [-1.10006, 1.03255, 0.338318],
              [-0.478579, 0.338318, 1.45178]])
print(check_definiteness(K))

Matrix:
 [[-0.984331 -1.10006  -0.478579]
 [-1.10006   1.03255   0.338318]
 [-0.478579  0.338318  1.45178 ]]
Eigenvalues: [-1.49999901  1.99999849  0.99999952]
Indefinite


In [11]:
# Concept Check 1 - Qn8
K = np.array([[-2.0353, 0.296916, -0.365128], 
              [0.296196, -1.10369, -0.074481],
              [-0.365128, -0.074481, -2.86101]])
print(check_definiteness(K))

Matrix:
 [[-2.0353    0.296916 -0.365128]
 [ 0.296196 -1.10369  -0.074481]
 [-0.365128 -0.074481 -2.86101 ]]
Eigenvalues: [-2.99999716 -1.9997948  -1.00020804]
Negative definite


## 2.1.2 - Lyapunov Function Definition